In [11]:
from sqlalchemy.orm import declarative_base, sessionmaker
from sqlalchemy.types import Integer, Text
from sqlalchemy.schema import ForeignKey, Column
from sqlalchemy.engine import create_engine

In [5]:
base = declarative_base()

In [7]:
base.metadata.tables

FacadeDict({})

In [8]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [10]:
dir(base)

['__abstract__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_sa_registry',
 'metadata',
 'registry']

In [40]:
base.registry.dispose() # base에 등록된 Class만 날림
# base.metadata.clear() # meta에 등록된 Table 객체를 날림

# 둘 다 서버와 상관없다

In [63]:
class User(base):
    __tablename__='USER'
    __table_args__= {'extend_existing':True}

    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text, nullable=False)

In [48]:
base.metadata.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)})

In [49]:
user1 = User(name='사람1')

In [50]:
user1.pk, user1.name # Class->Instance 값 조회 (DB와 상관 X)
# pk값 없음

(None, '사람1')

In [51]:
base.metadata.create_all(engine)
# Class(base) 선언 -> Metadata(Table 객체 등록) -> engine.dialenct가 Table객체를 SQL 변환
# engine.connect SQLite 접속 -> SQL 실행

2025-03-07 09:39:23,733 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 09:39:23,734 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("USER")
2025-03-07 09:39:23,734 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:39:23,735 INFO sqlalchemy.engine.Engine COMMIT


In [109]:
session = sessionmaker()
sess = session()
sess.bind = engine
#              주시
#instance <=========> 물리적 DB

In [21]:
s = sess()

In [27]:
s.bind = engine

In [28]:
s.add(user1)

In [29]:
s.dirty, s.is_modified(user1)
# session이 db에 기록/수정 차이점   session이 관리중인 객체가 변화가 있는지

(IdentitySet([]), True)

In [30]:
s.add(User(name='사람2'))

In [53]:
s.commit()

In [43]:
#s.rollback()
s.commit()

In [54]:
user1.pk

In [ ]:
result = sess.query(user).all()

In [ ]:
result[1].pk, result[1].name

In [ ]:
sess.dirty

In [ ]:
result[1].name = '바뀐 사람2'

In [ ]:
sess.dirty

In [ ]:
sess.commit()

In [55]:
result[1].name

NameError: name 'result' is not defined

In [ ]:
result[0].name, result[1].name

In [ ]:
result[0].name, result[1].name
# db까지 갈 필요가 없다

In [58]:
s.close()
base.registry.dispose()
base.metadata.clear()

In [68]:
sess.close_all

<bound method _SessionClassMethods.close_all of <class 'sqlalchemy.orm.session.sessionmaker'>>

In [75]:
sess.bind = engine

In [76]:
base.metadata.reflect(engine)

2025-03-07 09:47:49,277 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 09:47:49,278 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 09:47:49,278 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:47:49,279 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 09:47:49,279 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:47:49,279 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("USER")
2025-03-07 09:47:49,279 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 09:47:49,279 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-03-07 09:47:49,280 INFO sqlalchemy.engine.Engine [raw sql] ('USER',)
2025-03-07 09:47:49,280 INFO sqlalchemy.engine.Engine 

In [77]:
base.metadata.tables

FacadeDict({'USER': Table('USER', MetaData(), Column('PK', Integer(), table=<USER>, primary_key=True, nullable=False), Column('NAME', Text(), table=<USER>, nullable=False), schema=None)})

In [78]:
class NewUser(base):
    __table__ = base.metadata.tables['USER']

    NO = base.metadata.tables['USER'].c['PK']
    NAME = base.metadata.tables['USER'].c['NAME']

    def print(self):
        print(self.NO, self.NAME)

/var/folders/r0/ymsr5pxj0ld3f7szkbfyf__40000gn/T/ipykernel_44990/239469531.py:1: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.NewUser, and will be replaced in the string-lookup table.
  class NewUser(base):


In [79]:
r = sess.query(NewUser).all()

AttributeError: 'sessionmaker' object has no attribute 'query'

In [ ]:
r[0].print(), r[1].print()

In [80]:
sess.dirty

AttributeError: 'sessionmaker' object has no attribute 'dirty'

In [129]:
sess.close()
base.registry.dispose()
base.metadata.clear()
engine.connect().close()

In [146]:
engine = create_engine('sqlite:///coffee.db', echo = True)
sess.bind = engine
base.metadata.reflect(engine)

2025-03-07 10:49:01,948 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 10:49:01,948 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 10:49:01,948 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 10:49:01,949 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2025-03-07 10:49:01,949 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 10:49:01,949 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("album")
2025-03-07 10:49:01,949 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 10:49:01,950 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("artist")
2025-03-07 10:49:01,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 10:49:01,950 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("city")
2025-03-07 10:49:01,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07

In [84]:
class City(base):
    __table__ = base.metadata.tables['city']
class Supplier(base):
    __table__ = base.metadata.tables['supplier']
class Part(base):
    __table__ = base.metadata.tables['part']
class Sells(base):
    __table__ = base.metadata.tables['sells']

ArgumentError: Mapper Mapper[Sells(sells)] could not assemble any primary key columns for mapped table 'sells'

In [145]:
sess.close()
base.registry.dispose()
base.metadata.clear()
engine.connect().close()

In [133]:
engine = create_engine('sqlite:///:memory:', echo=True)
sess.bind = engine

In [134]:
class City(base):
    __tablename__ = 'city'
    cno = Column('cno', Integer, primary_key=True)
    name = Column('name', Text)

class Supplier(base):
    __tablename__ = 'supplier'
    sno = Column('sno', Integer, primary_key=True)
    name = Column('name', Text)
    cno = Column('cno', Integer, ForeignKey('city.cno'), nullable=False)

class Part(base):
    __tablename__ = 'part'
    cno = Column('pno', Integer, primary_key=True)
    name = Column('name', Text)

class Sells(base):
    __tablename__ = 'sells'
    no = Column('no', Integer, primary_key=True)
    sno = Column('sno', Integer, ForeignKey('supplier.sno'), nullable=False)
    cno = Column('cno', Integer, ForeignKey('part.cno'), nullable=False)
    price = Column('price', Integer, default=0)

In [135]:
city = list()
for c in ['성북구', '강북구', '아무구']:
    city.append(City(name=c))
sess.add_all(city)

In [ ]:
s

In [136]:
sess.commit()

2025-03-07 10:32:24,346 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 10:32:24,347 INFO sqlalchemy.engine.Engine INSERT INTO city (name) VALUES (?) RETURNING cno
2025-03-07 10:32:24,347 INFO sqlalchemy.engine.Engine [generated in 0.00005s (insertmanyvalues) 1/3 (ordered; batch not supported)] ('성북구',)
2025-03-07 10:32:24,348 INFO sqlalchemy.engine.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: city
[SQL: INSERT INTO city (name) VALUES (?) RETURNING cno]
[parameters: ('성북구',)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [116]:
city1 = sess.query(City).filter(City.name=='성북구').one()
city2 = sess.query(City).filter(City.name=='강북구').one()
city3 = sess.query(City).filter(City.name=='아무구').one()

PendingRollbackError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (sqlite3.OperationalError) no such table: city
[SQL: INSERT INTO city (name) VALUES (?) RETURNING cno]
[parameters: ('성북구',)]
(Background on this error at: https://sqlalche.me/e/20/e3q8) (Background on this error at: https://sqlalche.me/e/20/7s2a)

In [ ]:
slist = [Supplier(name='지점1', cno=city1.cno),
Supplier(name='지점2', cno=city1.cno),
Supplier(name='지점1', cno=city1.cno),
Supplier(name='지점2', cno=city1.cno),
Supplier(name='지점1', cno=city1.cno)]

In [ ]:
slist[0].sno
# Query X, Session에 등록했기 때문에 객체를 그냥 호출

In [ ]:
plite = [part(name='아이스 아메리카노'),
part(name='아이스티'),
part(name='음료1'),
part(name='음료2')]

sess.add_all(plist)
sess.commit()

In [ ]:
sess.add_all([Sells(sno=slist[0].sno, pno=plist[0].pno, price=10),
Sells(sno=slist[0].sno, pno=plist[0].pno, price=10),
Sells(sno=slist[0].sno, pno=plist[0].pno, price=10),
Sells(sno=slist[0].sno, pno=plist[0].pno, price=10),
])
# db에 insert까지만 한 상태, 실제 객체에는 값 x
sess.commit()

In [139]:
from sqlalchemy.orm import relationship

In [ ]:
#DB구조 안바뀜, Table 안 바꿈, mapped class 0
#SQLite X, metadata.table X, base.registry 초기화

In [147]:
class City(base):
    __table__ = base.metadata.tables['city']
    slist = relationship('Supplier', back_populates='city', uselist=True)

class Supplier(base):
    __table__ = base.metadata.tables['supplier']
    city = relationship('City', back_populates='slist')

In [150]:
c = sess.query(City).all()[0]

InvalidRequestError: One or more mappers failed to initialize - can't proceed with initialization of other mappers. Triggering mapper: 'Mapper[City(city)]'. Original exception was: list index out of range

In [204]:
sess.close()
engine.connect().close()
base.metadata.clear()
base.registry.dispose()

In [205]:
engine = create_engine('sqlite:///:memory:', echo=True)

In [206]:
sess.bind = engine

In [207]:
class Post(base):
    __tablename__ = 'post'
    pk = Column('pk', Integer, primary_key=True)
    content = Column('content', Text, default='')
    tags = relationship('PH', back_populates='post', uselist=True)

class Hashtag(base):
    __tablename__ = 'hashtag'
    pk = Column('pk', Integer, primary_key=True)
    name = Column('name', Text, default='')
    count = Column('count', Integer, default=0)
    posts = relationship('PH', back_populates='tag', uselist=True)

class PH(base):
    __tablename__ = 'ph'
    pk = Column('pk', Integer, primary_key=True)
    pfk = Column('pfk', Integer, ForeignKey('post.pk'))
    hfk = Column('hfk', Integer, ForeignKey('hashtag.pk'))
    post = relationship('Post', back_populates='tags')
    tag = relationship('Hashtag', back_populates='posts')

In [208]:
base.metadata.create_all(engine)

2025-03-07 11:35:08,524 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 11:35:08,524 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("post")
2025-03-07 11:35:08,525 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 11:35:08,525 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("post")
2025-03-07 11:35:08,525 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 11:35:08,526 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("hashtag")
2025-03-07 11:35:08,526 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 11:35:08,526 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("hashtag")
2025-03-07 11:35:08,526 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 11:35:08,526 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ph")
2025-03-07 11:35:08,527 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 11:35:08,527 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ph")
2025-03-07 11:35:08,527 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-07 11:3

In [209]:
p1 = Post(content='내용1')
sess.add(p1)

In [210]:
sess.add(p1)
sess.commit()

2025-03-07 11:35:11,923 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 11:35:11,924 INFO sqlalchemy.engine.Engine INSERT INTO post (content) VALUES (?)
2025-03-07 11:35:11,924 INFO sqlalchemy.engine.Engine [generated in 0.00031s] ('내용1',)
2025-03-07 11:35:11,924 INFO sqlalchemy.engine.Engine COMMIT


In [211]:
p1.pk

2025-03-07 11:35:12,839 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 11:35:12,840 INFO sqlalchemy.engine.Engine SELECT post.pk AS post_pk, post.content AS post_content 
FROM post 
WHERE post.pk = ?
2025-03-07 11:35:12,840 INFO sqlalchemy.engine.Engine [generated in 0.00031s] (1,)


1

In [212]:
h1 = Hashtag(name='해시태그1')
sess.add(h1)
sess.commit()

2025-03-07 11:35:14,870 INFO sqlalchemy.engine.Engine INSERT INTO hashtag (name, count) VALUES (?, ?)
2025-03-07 11:35:14,870 INFO sqlalchemy.engine.Engine [generated in 0.00050s] ('해시태그1', 0)
2025-03-07 11:35:14,871 INFO sqlalchemy.engine.Engine COMMIT


In [198]:
p1.tags.append(PH(pfk=p1.pk, hfk=Hashtag(name='해시태그1').pk))
# Post.tags[] -> backref = PH <- Hashtag

2025-03-07 11:31:46,120 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 11:31:46,121 INFO sqlalchemy.engine.Engine SELECT post.pk AS post_pk, post.content AS post_content 
FROM post 
WHERE post.pk = ?
2025-03-07 11:31:46,121 INFO sqlalchemy.engine.Engine [cached since 60.03s ago] (1,)
2025-03-07 11:31:46,122 INFO sqlalchemy.engine.Engine SELECT ph.pk AS ph_pk, ph.pfk AS ph_pfk, ph.hfk AS ph_hfk 
FROM ph 
WHERE ? = ph.pfk
2025-03-07 11:31:46,122 INFO sqlalchemy.engine.Engine [generated in 0.00021s] (1,)


In [199]:
sess.commit()

2025-03-07 11:31:53,315 INFO sqlalchemy.engine.Engine INSERT INTO ph (pfk, hfk) VALUES (?, ?)
2025-03-07 11:31:53,316 INFO sqlalchemy.engine.Engine [generated in 0.00057s] (1, None)
2025-03-07 11:31:53,316 INFO sqlalchemy.engine.Engine COMMIT


In [202]:
sess.query(PH).all()[0].fpk

ProgrammingError: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 6214709248 and this is thread id 8229144640.
[SQL: SELECT ph.pk AS ph_pk, ph.pfk AS ph_pfk, ph.hfk AS ph_hfk 
FROM ph]
[parameters: [{}]]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
ph1.post = p1
ph1.tag = Hashtag(name='해시태그2')

In [ ]:
sess.add(ph1)
sess.commit()

In [ ]:
ph1.post.content, ph1.tag.name

In [ ]:
p1= sess.query(Post).filter(Post.pk == 1).one()

In [ ]:
p1.tags[1].tag.name

In [213]:
ph1 = PH()

In [214]:
ph1.post = Post(content='내용1')
ph1.tag = Hashtag(name='태그1')

In [215]:
sess.add(ph1)
sess.commit()

2025-03-07 11:36:03,675 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-07 11:36:03,676 INFO sqlalchemy.engine.Engine INSERT INTO hashtag (name, count) VALUES (?, ?)
2025-03-07 11:36:03,676 INFO sqlalchemy.engine.Engine [cached since 48.81s ago] ('태그1', 0)
2025-03-07 11:36:03,676 INFO sqlalchemy.engine.Engine INSERT INTO post (content) VALUES (?)
2025-03-07 11:36:03,676 INFO sqlalchemy.engine.Engine [cached since 51.75s ago] ('내용1',)
2025-03-07 11:36:03,677 INFO sqlalchemy.engine.Engine INSERT INTO ph (pfk, hfk) VALUES (?, ?)
2025-03-07 11:36:03,677 INFO sqlalchemy.engine.Engine [generated in 0.00020s] (2, 2)
2025-03-07 11:36:03,678 INFO sqlalchemy.engine.Engine COMMIT


In [216]:
ph1.post.content, ph1.tag.name
# Insert Post 내용
# Insert Hashtag 태그
# Insert PH 내용,태그의 pk

ProgrammingError: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 6214709248 and this is thread id 8229144640.
[SQL: SELECT ph.pk AS ph_pk, ph.pfk AS ph_pfk, ph.hfk AS ph_hfk 
FROM ph 
WHERE ph.pk = ?]
[parameters: [{'pk_1': 1}]]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [217]:
p1.content, len(p1.tags), p1.tags[0].tag.name

ProgrammingError: (sqlite3.ProgrammingError) SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 6214709248 and this is thread id 8229144640.
[SQL: SELECT post.pk AS post_pk, post.content AS post_content 
FROM post 
WHERE post.pk = ?]
[parameters: [{'pk_1': 1}]]
(Background on this error at: https://sqlalche.me/e/20/f405)